In [1]:
import getpass
import os
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

os.environ["TAVILY_API_KEY"] = "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"
os.environ['ATHINA_API_KEY'] = "IhrJrr0krTMRA9ogqi5aaD4ZuYuvMcdG"

os.environ['SERPAPI_API_KEY']= "d79f01166cee047474b994092fd5f2462c2461490acf51ad91bf48ef10f46c28"


INFERENCE_SERVER_URL = "http://localhost:8989"
# MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
# MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
MODEL_NAME = "Qwen/Qwen3-14B"
API_KEY= "alanliuxiang"

llm = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base= f"{INFERENCE_SERVER_URL}/v1",
    model_name=MODEL_NAME,
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)


In [2]:
# # 读取系统变量
# from dotenv import load_dotenv
# load_dotenv()  

# # 初始化客户端
# from openai import OpenAI
# client = OpenAI()

# 定义检索鲜花库存的函数
import json
def get_flower_inventory(city):
    """获取指定城市的鲜花库存"""
    if "北京" in city:
        return json.dumps({"city": "北京", "inventory": "玫瑰: 100, 郁金香: 150"})
    elif "上海" in city:
        return json.dumps({"city": "上海", "inventory": "百合: 80, 康乃馨: 120"})
    elif "深圳" in city:
        return json.dumps({"city": "深圳", "inventory": "向日葵: 200, 玉兰: 90"})
    else:
        return json.dumps({"city": city, "inventory": "未知"})

# 工具（也就是函数）的元数据
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_flower_inventory",
            "description": "获取指定城市的鲜花库存",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "城市名称，例如：北京、上海或深圳"
                    }
                },
                "required": ["city"]
            }
        }
    }
]

# 第一次对话的Message
messages = [{"role": "user", "content": "北京、上海和深圳的鲜花库存是多少？"}]
print("message:", messages)

llm_with_tools = llm.bind_tools([get_flower_inventory])
# 第一次对话的返回结果

first_response = llm_with_tools.invoke(messages)

# first_response = llm.chat.completions.create(
#     model="Qwen/Qwen3-14B",
#     messages=messages,
#     tools=tools,
#     tool_choice="auto"
# )


print("first_response:", first_response)
# response_message = first_response.choices[0].message
# tool_calls = response_message.tool_calls

# 如果返回结果要求用Function Call，就调用函数，并把函数的查询结果附加到消息中
# if tool_calls:
#     messages.append(response_message)
#     for tool_call in tool_calls:
#         function_name = tool_call.function.name
#         function_args = json.loads(tool_call.function.arguments)
#         function_response = get_flower_inventory(
#             city=function_args.get("city")
#         )
#         messages.append(
#             {
#                 "tool_call_id": tool_call.id,
#                 "role": "tool",
#                 "name": function_name,
#                 "content": function_response,
#             }
#         )
# print("message:", messages)

# 用有了库存查询结果的Message再来一次对话
# second_response = client.chat.completions.create(
#     # model="gpt-3.5-turbo-0125",
#     model="Qwen/Qwen3-14B",
#     messages=messages
#     )
second_response = llm_with_tools.invoke(messages)
print("second_response:", second_response)


message: [{'role': 'user', 'content': '北京、上海和深圳的鲜花库存是多少？'}]
<think>
好的，用户问的是北京、上海和深圳三个城市的鲜花库存。我需要调用get_flower_inventory这个函数来获取每个城市的库存数据。首先，确定函数需要的参数是城市名称，所以我要分别对这三个城市调用三次函数。然后，把每次返回的结果整理起来，用清晰的方式呈现给用户。可能用户需要比较这三个城市的库存情况，或者了解各个城市的供应情况，所以确保每个城市的库存数据都准确无误地展示出来。如果有任何城市的数据获取失败，可能需要提示用户，但假设函数都能正确返回的话，就按正常流程处理。
</think>

<tool_call>
{"name": "get_flower_inventory", "arguments": {"city": "北京"}}
</tool_call>
<tool_call>
{"name": "get_flower_inventory", "arguments": {"city": "上海"}}
</tool_call>
<tool_call>
{"name": "get_flower_inventory", "arguments": {"city": "深圳"}}
</tool_call>first_response: content='<think>\n好的，用户问的是北京、上海和深圳三个城市的鲜花库存。我需要调用get_flower_inventory这个函数来获取每个城市的库存数据。首先，确定函数需要的参数是城市名称，所以我要分别对这三个城市调用三次函数。然后，把每次返回的结果整理起来，用清晰的方式呈现给用户。可能用户需要比较这三个城市的库存情况，或者了解各个城市的供应情况，所以确保每个城市的库存数据都准确无误地展示出来。如果有任何城市的数据获取失败，可能需要提示用户，但假设函数都能正确返回的话，就按正常流程处理。\n</think>\n\n<tool_call>\n{"name": "get_flower_inventory", "arguments": {"city": "北京"}}\n</tool_call>\n<tool_call>\n{"name": "get_flower_inventory", "ar